# Dynamic repetition rate execution

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.40.1
```
</details>

When a circuit is executed on an IBM&reg; quantum processing unit (QPU), an implicit reset is typically inserted at the beginning of the circuit to ensure the qubits are initialized to zero. However, the reset process is not perfect, leading to state preparation errors. To alleviate the error, the system also inserts repetition delay time (or `rep_delay`) between circuits. Each backend has a different default `rep_delay`, but it's usually longer than T1 to allow the environment to reset the qubits. The default `rep_delay` can be queried by running `backend.default_rep_delay`.

All IBM QPUs use dynamic repetition rate execution, which allows you to change the `rep_delay` for each job. Circuits you submit in a primitive job are batched together for execution on the QPU. These circuits are executed by iterating over the circuits for each shot requested; the execution is column-wise over a matrix of circuits and shots, as illustrated in the following figure.

![The first column represents shot 0.  The circuits are run in order from 0 through 3.  The second column represents shot 1.  The circuits are run in order from 0 through 3.  The remaining columns follow the same pattern. ](/docs/images/guides/repetition-rate-execution/circuits_shots_matrix1.avif "Column-wise execution matrix")


Because `rep_delay` is inserted between circuits, each shot of the execution encounters this delay. Therefore, lowering the `rep_delay` decreases the total QPU execution time, but at the expense of increased state preparation error rate, as can be seen in the following image:

![This image shows that as the `rep_delay` value is lowered, the state preparation error rate increases.](/docs/images/guides/repetition-rate-execution/rep_delay.avif "Repetition delay versus error rate")


Note that while circuits in a primitive job are batched together for QPU execution, there is no guarantee on the order the circuits from PUBs are executed. Thus, even though you submit `pubs=[pub1, pub2]`, there is no guarantee the circuits from `pub1` will run before those from `pub2`. There is also no guarantee that circuits from the same job would run as a single batch on the QPU.

## Specify rep_delay for a primitive job

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

service = QiskitRuntimeService()

# Make sure your backend supports it
backend = service.least_busy(
    operational=True, min_num_qubits=100, dynamic_reprate_enabled=True
)

# Determine the allowable range
backend.rep_delay_range
sampler = Sampler(mode=backend)

# Specify a value in the supported range
sampler.options.execution.rep_delay = 0.0005

<Admonition type="tip" title="The init_qubits flag">

The `init_qubits` flag, set as a [primitive execution option,](/docs/api/qiskit-ibm-runtime/options-execution-options-v2) controls whether qubits are reset to the zero state at the start of each circuit. Its default value is `True`, indicating that the qubits should be reset. If `False`, the qubits will begin in the final state from the previous shot, and you must manually insert [resets](/docs/api/qiskit/circuit#qiskit.circuit.Reset) if you want to reset them to the zero state. If a job consists of multiple circuits, then the shots are executed in a "round-robin" fashion. That is, each circuit will be executed in sequence to obtain one shot from each circuit. This process is then repeated until the requested number of shots has been obtained from all circuits.
</Admonition>

## Next steps

<Admonition type="tip" title="Recommendations">
  - Try an example in the [Quantum approximate optimization algorithm (QAOA)](/docs/tutorials/quantum-approximate-optimization-algorithm) tutorial.
  - Review how to [get started with primitives.](./get-started-with-primitives)
</Admonition>